In [1]:
import pandas as pd
import numpy as np

In [2]:
from preprocessing import filter_and_extract_lemma, one_hot_encode, one_hot_decode
from preprocessing import ExtendedTokenizer

In [3]:
train = pd.read_csv('data/emotion_dataset/train.txt', sep=';', names=['Text', 'Emotion'])
validation = pd.read_csv('data/emotion_dataset/val.txt', sep=';', names=['Text', 'Emotion'])
test = pd.read_csv('data/emotion_dataset/test.txt', sep=';', names=['Text', 'Emotion'])

In [4]:
train['Text'] = train['Text'].apply(filter_and_extract_lemma)
validation['Text'] = validation['Text'].apply(filter_and_extract_lemma)
test['Text'] = test['Text'].apply(filter_and_extract_lemma)

In [5]:
tokenizer = ExtendedTokenizer(num_words=15000, lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(train['Text'].values)

In [6]:
x_train = tokenizer.texts_to_padded_sequences(train['Text'].values)
x_val = tokenizer.texts_to_padded_sequences(validation['Text'].values)
x_test = tokenizer.texts_to_padded_sequences(test['Text'].values)

In [7]:
y_train = one_hot_encode(train['Emotion'])
y_val = one_hot_encode(validation['Emotion'])
y_test = one_hot_encode(test['Emotion'])